In [1]:
#FTSE 100 Constituents

tickers = (
'III.L',
'ADM.L',
'AAF.L',
'AAL.L',
'ANTO.L',
'AHT.L',
'ABF.L',
'AZN.L',
'AUTO.L',
'AV.L',
'BME.L',
'BA.L',
'BARC.L',
'BDEV.L',
'BEZ.L',
'BKG.L',
'BP.L',
'BATS.L',
'BT-A.L',
'BNZL.L',
'BRBY.L',
'CNA.L',
'CCH.L',
'CPG.L',
'CTEC.L',
'CRDA.L',
'DCC.L',
# 'DPH.L',
'DGE.L',
'DPLM.L',
'EDV.L',
'ENT.L',
'EXPN.L',
'FCIT.L',
'FLTR.L',
'FRAS.L',
'FRES.L',
'GLEN.L',
'GSK.L',
'HLN.L',
'HLMA.L',
'HL.L',
'HIK.L',
'HWDN.L',
'HSBA.L',
'IHG.L',
'IMI.L',
'IMB.L',
'INF.L',
'IAG.L',
'ITRK.L',
'JD.L',
'KGF.L',
'LAND.L',
'LGEN.L',
'LLOY.L',
'LSEG.L',
'MNG.L',
'MKS.L',
'MRO.L',
'MNDI.L',
'NG.L',
'NWG.L',
'NXT.L',
'OCDO.L',
'PSON.L',
'PSH.L',
'PHNX.L',
'PRU.L',
'RKT.L',
'REL.L',
'RTO.L',
'RMV.L',
'RIO.L',
'RR.L',
'RS1.L',
'SGE.L',
'SBRY.L',
'SDR.L',
'SMT.L',
'SGRO.L',
'SVT.L',
'SHEL.L',
'SMDS.L',
'SMIN.L',
'SN.L',
'SKG.L',
'SPX.L',
'SSE.L',
'STAN.L',
'STJ.L',
'TW.L',
'TSCO.L',
'ULVR.L',
'UU.L',
'UTG.L',
'VOD.L',
'WEIR.L',
'WTB.L',
'WPP.L',
)

In [5]:
#Required Libraries

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

#Retrieve stock data from Yahoo Finance

def get_stock_data(symbol):
    try:
        stock_data = yf.download(symbol, period='1Y')
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function that uses LSTM model to predict next day prices

def predict_next_day_closing_price(ticker):
    data = get_stock_data(ticker)
    if data is None:
        return None, None

    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Adj Close'].values.reshape(-1, 1))

    # Prepare the data for LSTM
    def create_dataset(dataset, time_steps=1):
        X, y = [], []
        for i in range(len(dataset) - time_steps):
            a = dataset[i:(i + time_steps), 0]
            X.append(a)
            y.append(dataset[i + time_steps, 0])
        return np.array(X), np.array(y)

    time_steps = 10  # You can experiment with the number of time steps
    X, y = create_dataset(scaled_data, time_steps)

    # Reshape the data for LSTM input
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=100, batch_size=32)

    # Make predictions for the next day
    last_sequence = scaled_data[-time_steps:]
    last_sequence = last_sequence.reshape((1, time_steps, 1))
    predicted_value = model.predict(last_sequence)
    predicted_price = scaler.inverse_transform(predicted_value.reshape(-1, 1))

    # Get the latest closing price
    latest_close_price = data['Adj Close'].iloc[-1]

    return predicted_price[0, 0], latest_close_price

# Create a list to store results
results = []

# Predict next day closing prices for each ticker and store in empty results above
for ticker in tickers:
    predicted_price, latest_close_price = predict_next_day_closing_price(ticker)
    if predicted_price is not None:
        results.append({'Ticker': ticker, 'PredictedPrice': predicted_price, 'LatestClosePrice': latest_close_price})

# Create a DataFrame from the results list
result_df = pd.DataFrame(results)

# Display the DataFrame
print(result_df)

[*********************100%%**********************]  1 of 1 completed
Epoch 1/100
8/8 [==============================] - 1s 6ms/step - loss: 0.1453
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0220
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0126
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0111
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0053
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0040
Epoch 7/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0037
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0028
Epoch 10/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0027
Epoch 11/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 12/100
8/8 [==============================] - 0s 4ms/step - loss:

8/8 [==============================] - 0s 3ms/step - loss: 0.0213
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0172
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0162
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0154
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0150
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0146
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0143
Epoch 11/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0141
Epoch 12/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0145
Epoch 13/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0139
Epoch 14/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0133
Epoch 15/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0130
Epoch 16/100
8/8 [==============================] - 0s 3ms/step - loss

8/8 [==============================] - 0s 3ms/step - loss: 0.0048
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0044
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0044
Epoch 11/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0043
Epoch 12/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 13/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0046
Epoch 14/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0043
Epoch 15/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 16/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 17/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 18/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 19/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 20/100
8/8 [==============================] - 0s 3ms/step - 

8/8 [==============================] - 0s 3ms/step - loss: 0.0051
Epoch 13/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0051
Epoch 14/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0051
Epoch 15/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0050
Epoch 16/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0049
Epoch 17/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0049
Epoch 18/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0049
Epoch 19/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0048
Epoch 20/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0048
Epoch 21/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0047
Epoch 22/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0046
Epoch 23/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0046
Epoch 24/100
8/8 [==============================] - 0s 3ms/step -

8/8 [==============================] - 0s 3ms/step - loss: 0.0100
Epoch 17/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0099
Epoch 18/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0097
Epoch 19/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0096
Epoch 20/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0095
Epoch 21/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0093
Epoch 22/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0092
Epoch 23/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0090
Epoch 24/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0090
Epoch 25/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0086
Epoch 26/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0086
Epoch 27/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0083
Epoch 28/100
8/8 [==============================] - 0s 3ms/step -

...

8/8 [==============================] - 0s 3ms/step - loss: 0.0085
Epoch 77/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0098
Epoch 78/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0091
Epoch 79/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0080
Epoch 80/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0084
Epoch 81/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0084
Epoch 82/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0081
Epoch 83/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0079
Epoch 84/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0077
Epoch 85/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0079
Epoch 86/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0077
Epoch 87/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0075
Epoch 88/100
8/8 [==============================] - 0s 3ms/step -

8/8 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 81/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0039
Epoch 82/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0040
Epoch 83/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 84/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 85/100
8/8 [==============================] - 0s 6ms/step - loss: 0.0039
Epoch 86/100
8/8 [==============================] - 0s 4ms/step - loss: 0.0041
Epoch 87/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0045
Epoch 88/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0044
Epoch 89/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0042
Epoch 90/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0039
Epoch 91/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0039
Epoch 92/100
8/8 [==============================] - 0s 3ms/step -

8/8 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 85/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 86/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 87/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0035
Epoch 88/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0034
Epoch 89/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0038
Epoch 90/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 91/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 92/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 93/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 94/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 95/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0032
Epoch 96/100
8/8 [==============================] - 0s 3ms/step -


1 Failed download:
['IHG.L']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [6]:
# Export results as CSV file to preferred location

result_df.to_csv(r'#yourlocationpath/ftse_predicted.csv')